<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Alexander_Valle/IA025_Alexander_Valle_Aula9_Exerc%C3%ADcioR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Rolan Alexander Valle Rey Sánchez RA230254'
print(f'Meu nome é {nome}')

Meu nome é Rolan Alexander Valle Rey Sánchez RA230254


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.5 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  1 11:50:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.x = []
        self.y = []
        starid=tokenize(text='[CLS]', tokenizer=tokenizer)
        #print('starid',starid)
        padid=tokenizer.pad_token_id
        #print('padid',padid)
        for text in texts:
          #print('text: ',text)
          tokens = tokenize(text, tokenizer)
          #print(len(tokens),max_seq_length)
          if len(tokens) <= max_seq_length:
            self.x.append(starid+tokens[0:len(tokens)]+[ padid for k in range(len(tokens)+1,max_seq_length)])
            self.y.append(tokens[0:len(tokens)]+[ padid for k in range(len(tokens),max_seq_length)])
          else:
            for i in range(len(tokens)+1 - max_seq_length):
              self.x.append(starid+tokens[i:i +max_seq_length-1])
              self.y.append(tokens[i:i +max_seq_length])
        #print("x,y  ",self.x,' ',self.y)
        self.x = torch.tensor(self.x, dtype=torch.long)
        self.y = torch.tensor(self.y, dtype=torch.long)

    def __len__(self):
        # Escreva seu código aqui
        return len(self.x)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        return self.x[idx], self.y[idx]


## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [ ]:
dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)


In [ ]:
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')


Passou no assert de tamanho do dataset.


In [ ]:
first_batch_input, first_batch_target = next(iter(dummy_loader))
first_batch_input, first_batch_target

(tensor([[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
         [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]]),
 tensor([[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
         [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]]))

In [ ]:
first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-01 11:50:10--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.12.128, 108.177.13.128, 172.217.193.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.12.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   192MB/s    in 6.3s    

2022-06-01 11:50:17 (187 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [ ]:
# Load datasets
max_seq_length = 9

train_examples = 12000
valid_examples = 4000
test_examples = 4000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 20000 lines.


In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 13962947
valid examples: 4305181
test examples: 4067492


In [ ]:
"""
problems with memory management when use mask transformation
to get the attatention result
"""
import torch.nn.functional as F
#def __init__(self, vocab_size, context_size, embedding_dim):
# inspiration from http://www.peterbloem.nl/blog/transformers
# https://www.kaggle.com/code/arunmohan003/transformer-from-scratch-using-pytorch/notebook

class LanguageModel2(torch.nn.Module):
  def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
    """Implements the Self-attention, decoder-only.
    Args: vocab_size (int): Size of the input vocabulary.
          max_seq_length (int): Size of the sequence to consider as context for prediction.
          dim (int): Dimension of the embedding layer for each word in the context.
          n_layers (int): number of self-attention layers.
          pad_token_id (int): id of the pad token that will be ignored in the attention.
    """
    super().__init__()
    self.D=dim#embedding_dim  
    self.n_layes=n_layers
    self.L=max_seq_length
    self.pad_token_id=pad_token_id
    #self.L=context_size  
    # These compute the queries, keys and values for all  
    self.W_keys    = nn.Linear(dim, dim, bias=False)
    self.W_queries = nn.Linear(dim, dim, bias=False)
    self.W_values  = nn.Linear(dim, dim, bias=False)
    self.Wo  = nn.Linear(dim, dim, bias=False)

    self.C= nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)#word embeddings
    self.P= nn.Embedding(max_seq_length, dim, padding_idx=pad_token_id)# positional_embeddings of the words
    #self.heads =dim//context_size
    self.ff1= nn.Linear(dim, vocab_size)

    hsf=4#hidden size factor
    feedforward_dim=hsf*dim
    self.L1 = nn.Linear(dim, feedforward_dim)
    self.Drop1 =nn.Dropout(0.2)
    self.Drop2 =nn.Dropout(0.1)
    self.relu = nn.ReLU()#inplace=Truevocab_size
    self.L2 = nn.Linear(feedforward_dim,dim, bias=False)
    # Maps the final output sequence to class logits
    self.ff = nn.Sequential(self.L1 ,self.Drop1,self.relu,self.L2,self.Drop2)#feedforward
    self.debug=False
    self.norm1 = nn.LayerNorm(dim)
    self.norm2 = nn.LayerNorm(dim)
    self.tologits = nn.Linear(dim, vocab_size, bias = False)


  def get_mask(self, input):
    #https://www.programmersought.com/article/60757409558/
    batch_size = input.shape[0]#                device=mask.device,
    #triangular mask
    mask = torch.triu(torch.ones((self.L, self.L), dtype=torch.uint8),diagonal=1).to(device)
    mask = mask.unsqueeze(0).expand(batch_size, -1, -1)  # [B, L, L]
    mask = mask.masked_fill(input.unsqueeze(1) == self.pad_token_id, 0)#pedro Grengo
    mask = mask.masked_fill(input.unsqueeze(2) == self.pad_token_id, 0)
    return mask

  def attention_mask(self, x,mask):
    Q = self.W_queries(x) # shape = B, C, D#[:, -1, :].unsqueeze(1)
    K = self.W_keys(x) # shape = B, C, D
    V = self.W_values(x) # shape = B, C, D
    #Attention(Q,K,V)=softmax(QKT/sqrt(dk))V
    # scores shape: (B, 1, L)
    #print('x.shape:', x.shape)
    QKT=torch.matmul(Q, K.transpose(-2, -1))
    #print('QKT.shape:', QKT.shape)
    #QKT=torch.matmul(Q,torch.transpose(K,1,2))
    QKTmasked = QKT.masked_fill(mask.unsqueeze(1) == 0, -float("inf"))# Pedro Gengo
    #scores = QKTmasked/(self.D**0.5) 
    #probs = F.softmax(scores, dim=-1)
    probs = F.softmax(QKTmasked, dim=-1)
    A=torch.matmul(probs, V) #   shape: (B, 1, D)
    if self.debug:
      print('x.shape:', x.shape)
      print('QKTmasked.shape:', QKTmasked.shape)
      print('K.shape:', K.shape)
      print('V.shape:', V.shape)
      #print('scores.shape:', scores.shape)
      print('probs.shape:', probs.shape)
      print('Attention.shape:', A.shape)
    return A

  def TBlock(self, x,BS,mask):
    for k in range(self.n_layes):
      R=x
      #x = R+ self.Wo(self.attention(x))
      x = R+ self.Wo(self.attention_mask(x,mask))
    #print('x.shape',x.shape)
    E = self.norm1(x)
    #print('E.shape',E.shape)
    EF=E.view(BS,-1)# fron Larissa 
    #print('EF.shape',EF.shape)
    fedforward = self.ff(E) # B, V  
    #print('FF.shape',fedforward.shape)
    out=self.norm2(fedforward + E)
    #print('out.shape',out.shape)
    if self.debug:
      print('x.shape:', x.shape)
      #print('A.shape:', A.shape)
      print('E.shape',E.shape)
      print('EF.shape',EF.shape)
      print('FF.shape',fedforward.shape)
      print('out.shape',out.shape)
    return out

  def forward(self, inputs):
    """
    Args:
    inputs is a LongTensor of shape (batch_size, max_seq_length)
    Returns:
    logits of shape (batch_size, max_seq_length, vocab_size)
    """
    B = inputs.shape[0]# B: batch_size
    tokens = self.C(inputs)# generate token embeddings
    # generate position embeddings , L: max_seq_length        
    positions = self.P.weight # 
    X = tokens + positions
    #X = tokens
    #print('X.shape',X.shape)
    mask=self.get_mask(inputs)
    out=self.TBlock(X,B,mask)
    logits = self.tologits(out) # B, V
    #print('logits.shape:', logits.shape)      
    if self.debug:
      print('batch_size',B)
      print('tokens.size()',tokens.size())
      #print('positions.size()',positions.size())
      print('out.shape:', out.shape)
      print('logits.shape:', logits.shape)      
    return logits




"""
train_examples = 6000
valid_examples = 2000
test_examples = 2000

RuntimeError: CUDA out of memory. Tried to allocate 2.15 GiB (GPU 0; 15.90 GiB total capacity;
 14.76 GiB already allocated; 95.75 MiB free; 14.78 GiB reserved in total by PyTorch)
  If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.
  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
"""

In [ ]:
import torch.nn.functional as F
#def __init__(self, vocab_size, context_size, embedding_dim):
# inspiration from http://www.peterbloem.nl/blog/transformers
# https://www.kaggle.com/code/arunmohan003/transformer-from-scratch-using-pytorch/notebook

class LanguageModel(torch.nn.Module):
  def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
    """Implements the Self-attention, decoder-only.
    Args: vocab_size (int): Size of the input vocabulary.
          max_seq_length (int): Size of the sequence to consider as context for prediction.
          dim (int): Dimension of the embedding layer for each word in the context.
          n_layers (int): number of self-attention layers.
          pad_token_id (int): id of the pad token that will be ignored in the attention.
    """
    super().__init__()
    self.D=dim#embedding_dim  
    self.n_layes=n_layers
    self.L=max_seq_length
    self.pad_token_id=pad_token_id
    #self.L=context_size  
    # These compute the queries, keys and values for all  
    self.W_keys    = nn.Linear(dim, dim, bias=False)
    self.W_queries = nn.Linear(dim, dim, bias=False)
    self.W_values  = nn.Linear(dim, dim, bias=False)
    self.Wo  = nn.Linear(dim, dim, bias=False)

    self.C= nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)#word embeddings
    self.P= nn.Embedding(max_seq_length, dim, padding_idx=pad_token_id)# positional_embeddings of the words
    #self.heads =dim//context_size
    self.ff1= nn.Linear(dim, vocab_size)

    hsf=4#hidden size factor
    feedforward_dim=hsf*dim
    self.L1 = nn.Linear(dim, feedforward_dim)
    self.Drop1 =nn.Dropout(0.2)
    self.Drop2 =nn.Dropout(0.1)
    self.relu = nn.ReLU()#inplace=Truevocab_size
    self.L2 = nn.Linear(feedforward_dim,dim, bias=False)
    # Maps the final output sequence to class logits
    self.ff = nn.Sequential(self.L1 ,self.Drop1,self.relu,self.L2,self.Drop2)#feedforward
    self.debug=False
    self.norm1 = nn.LayerNorm(dim)
    self.norm2 = nn.LayerNorm(dim)
    self.tologits = nn.Linear(dim, vocab_size, bias = False)

  def attention(self, x):
    Q = self.W_queries(x) # shape = B, C, D#[:, -1, :].unsqueeze(1)
    K = self.W_keys(x) # shape = B, C, D
    V = self.W_values(x) # shape = B, C, D
    #Attention(Q,K,V)=softmax(QKT/sqrt(dk))V
    # scores shape: (B, 1, L)
    scores = torch.matmul(Q,torch.transpose(K,1,2))/(self.D**0.5) 
    probs = F.softmax(scores, dim=-1)
    A=torch.matmul(probs, V) #   shape: (B, 1, D)
    if self.debug:
      print('x.shape:', x.shape)
      print('Q.shape:', Q.shape)
      print('K.shape:', K.shape)
      print('V.shape:', V.shape)
      print('scores.shape:', scores.shape)
      print('probs.shape:', probs.shape)
      print('Attention.shape:', A.shape)
    return A

  def TBlock(self, x,BS):
    for k in range(self.n_layes):
      R=x
      x = R+ self.Wo(self.attention(x))
    #print('x.shape',x.shape)
    E = self.norm1(x)
    #print('E.shape',E.shape)
    EF=E.view(BS,-1)# fron Larissa 
    #print('EF.shape',EF.shape)
    fedforward = self.ff(E) # B, V  
    #print('FF.shape',fedforward.shape)
    out=self.norm2(fedforward + E)
    #print('out.shape',out.shape)
    if self.debug:
      print('x.shape:', x.shape)
      #print('A.shape:', A.shape)
      print('E.shape',E.shape)
      print('EF.shape',EF.shape)
      print('FF.shape',fedforward.shape)
      print('out.shape',out.shape)
    return out

  def forward(self, inputs):
    """
    Args:
    inputs is a LongTensor of shape (batch_size, max_seq_length)
    Returns:
    logits of shape (batch_size, max_seq_length, vocab_size)
    """
    B = inputs.shape[0]# B: batch_size
    tokens = self.C(inputs)# generate token embeddings
    # generate position embeddings , L: max_seq_length        
    positions = self.P.weight # 
    X = tokens + positions
    #X = tokens
    #print('X.shape',X.shape)
    out=self.TBlock(X,B)
    logits = self.tologits(out) # B, V
    #print('logits.shape:', logits.shape)      
    if self.debug:
      print('batch_size',B)
      print('tokens.size()',tokens.size())
      #print('positions.size()',positions.size())
      print('out.shape:', out.shape)
      print('logits.shape:', logits.shape)      
    return logits

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)



In [ ]:
sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5800482


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
        target: a LongTensor of shape (batch_size, seq_len)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              35291
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
#max_examples = 150_000_000
max_examples = 31_000_000


eval_every_steps = 10000
lr = 5e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=128,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 34557.38, valid ppl: 32856.39
10000 steps; 10240000 examples so far; train ppl: 2.43, valid ppl: 1.91
20000 steps; 20480000 examples so far; train ppl: 1.87, valid ppl: 1.85
30000 steps; 30720000 examples so far; train ppl: 1.83, valid ppl: 1.83


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 1.8287806239960884


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Ontem fui ao restaurante comer um prato delicioso, pedi'#  tem medo de água fria
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Ontem fui ao restaurante comer um prato delicioso, pedi restaurante
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um dia
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delic
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delicioso
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delicioso,
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delicioso, ped
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delicioso, pedi
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delicioso, pedi restaurante
Ontem fui ao restaurante comer um prato delicioso, pedi restaurante um diam delicioso, pedi restaurante um
Ontem fui ao restaurante comer um prato delicioso, pedi resta

In [ ]:
prompt = 'A grama da vizinho é sempre é sempre mais verde'# max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A grama da vizinho é sempre é sempre mais verde mais
A grama da vizinho é sempre é sempre mais verde mais da
A grama da vizinho é sempre é sempre mais verde mais da mais
A grama da vizinho é sempre é sempre mais verde mais da mais da
A grama da vizinho é sempre é sempre mais verde mais da mais da mais
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais da
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais da mais
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais da mais da
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais da mais da da
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais da mais da da da
A grama da vizinho é sempre é sempre mais verde mais da mais da mais da mais da mais da da da da
A grama da vi

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.